<a href="https://colab.research.google.com/github/chanseongparkk/BigDataAnalyst_practise/blob/main/%EB%8C%80%ED%95%99%EC%9B%90%EC%9E%85%ED%95%99%EC%98%88%EC%B8%A1(%ED%9A%8C%EA%B7%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 대학원 입학 예측(회귀)
* 예측할 값(target): "Chance of Admit "
* 평가: r2
* data(3개): t2-2-X_train, t2-2-y_train, t2-2-X_test
* 제출 형식(Serial No.-> id, 예측 값 -> target)

### Baseline
* 데이터 크기가 작아 크로스 밸리데이션만 확인했어요
* 타겟 피처에 공백이 있어요 "Chance of Admit "
* r2로 평가해요

In [ ]:
cd drive/My Drive/빅데이터분석기사 실기/대학원 입학 예측(회귀)

/content/drive/My Drive/빅데이터분석기사 실기/대학원 입학 예측(회귀)


In [ ]:
import pandas as pd

X_train = pd.read_csv("t2-2-X_train.csv")
X_test = pd.read_csv("t2-2-X_test.csv")
y_train = pd.read_csv("t2-2-y_train.csv")

### 시리얼넘버 따로 분리 후 저장 및 데이터셋에서 삭제
X_train_id = X_train["Serial No."]
X_test_id = X_test["Serial No."]
y_train_id = y_train["Serial No."]

X_train = X_train.drop(["Serial No."],axis=1)
X_test = X_test.drop(["Serial No."],axis=1)
y_train = y_train.drop(["Serial No."],axis=1)
# display(X_train.info(),X_test.info(),y_train.info())

### 데이터 결측치 확인

#print("X_train 결측치  ","\n\n", X_train.isna().sum(),"\n\n",
#     "X_test 결측치  ","\n\n", X_test.isna().sum()) # 결측치가 존재하지 않음

### 데이터 기초통계량 확인

#display(X_train.describe(),X_test.describe())

### Research 변수 범주형으로 변경(0: 경험X 1: 경험o)

X_train.Research = X_train.Research.astype("object")
X_test.Research = X_test.Research.astype("object")

### 데이터 변환
#print(X_train.skew(), X_train.kurt())
#데이터 변환이 필요없어 보인다.

### 데이터 스케일링

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train.iloc[:,:6] = scaler.fit_transform(X_train.iloc[:,:6])
X_test.iloc[:,:6] = scaler.transform(X_test.iloc[:,:6])

# display(X_train.describe(),X_test.describe(),X_train.info())

### 범주형 변수 더미화

X_train = pd.get_dummies(X_train,drop_first = True)
X_test = pd.get_dummies(X_test, drop_first = True)

# print(X_train.info(),X_test.info())

### train data valid data split

from sklearn.model_selection import train_test_split

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size = 0.3 ,random_state = 999)

# print(X_train2, X_val, y_train2, y_val)

# 모델링(RandomRorestRegressor)
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf = RandomForestRegressor(n_estimators = 300, max_depth = 5, min_samples_split = 10, random_state = 999)

rf.fit(X_train2,y_train2)

'''
print("rf train accuracy : ",rf.score(X_train2,y_train2),"\n",
      "rf train r2_score : ",r2_score(y_train2,rf.predict(X_train2)),"\n"
      "rf validation accuracy : ",rf.score(X_val,y_val),"\n",
      "rf validation r2_score : ",r2_score(y_val,rf.predict(X_val)))
      '''

# 모델링(XGBRegressor)

from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators = 300, max_depth = 5, subsample = 0.3,learning_rate = 0.01, random_srate = 999, objective = "reg:squarederror")
xgb.fit(X_train2, y_train2, eval_metric = "rmse")

'''
print("xgb train accuracy : ",xgb.score(X_train2,y_train2),"\n",
      "xgb train r2_score : ",r2_score(y_train2,xgb.predict(X_train2)),"\n",
      "xgb validation accuracy : ",xgb.score(X_val,y_val),"\n",
      "xgb validation r2_score : ",r2_score(y_val,xgb.predict(X_val)))
      '''

### 최종 제출용 데이터셋(rf, xgb 예측값 평균을 최종 예측값으로 사용)
pred = (xgb.predict(X_test) + rf.predict(X_test))/2


final = pd.concat([pd.DataFrame(X_test_id),pd.DataFrame(pred)],axis = 1)
final.columns = ["id", "target"]
# print(final)
# final.to_csv("004002325.csv",index = False)
pd.read_csv("004002325.csv")



,id,target
0,128,0.712650
1,76,0.696404
2,151,0.907028
3,102,0.650171
4,272,0.522736
...,...,...
75,252,0.719169
76,362,0.911891
77,338,0.912830
78,186,0.838984
